In [12]:
pip install pandas numpy faker



   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 882.6 kB/s eta 0:00:02
   ---------- ----------------------------- 0.5/2.0 MB 882.6 kB/s eta 0:00:02
   --------------- ------------------------ 0.8/2.0 MB 860.9 kB/s eta 0:00:02
   --------------------- ------------------ 1.0/2.0 MB 915.5 kB/s eta 0:00:02
   -------------------------- ------------- 1.3/2.0 MB 932.1 kB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 932.1 kB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 943.0 kB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 1.0 MB/s eta 0:00:00


In [13]:
#Import necessary Libraries
import pandas as pd
import numpy as np
from faker import Faker

#Generate Synthetic Data for Customers
fake = Faker()
np.random.seed(42)

n_customers = 500

customers = pd.DataFrame({
    "customer_id": range(1, n_customers + 1),
    "age": np.random.randint(18, 70, n_customers),
    "gender": np.random.choice(["Male", "Female"], n_customers),
    "country": np.random.choice(["UK", "Germany", "France", "Spain"], n_customers),
    "balance": np.round(np.random.uniform(50, 50000, n_customers), 2),
    "account_type": np.random.choice(["Basic", "Premium"], n_customers, p=[0.7, 0.3])
})



In [14]:
#Generate Synthetic Data for Products
products = pd.DataFrame({
    "product_id": range(1, n_customers + 1),
    "customer_id": customers["customer_id"],
    "product_name": np.random.choice(
        ["Savings Account", "Credit Card", "Loan", "Investment"], n_customers
    ),
    "product_category": np.random.choice(
        ["Savings", "Credit", "Loan", "Investment"], n_customers
    )
})


In [22]:
#Generate Transaction Table
from datetime import datetime, timedelta

n_transactions = 8000

transactions = pd.DataFrame({
    "transaction_id": range(1, n_transactions + 1),
    "customer_id": np.random.choice(customers["customer_id"], n_transactions),
    "transaction_date": [
        fake.date_between(start_date="-1y", end_date="today")
        for _ in range(n_transactions)
    ],
    "amount": np.round(np.random.uniform(5, 2000, n_transactions), 2),
    "transaction_type": np.random.choice(
        ["Card Payment", "Transfer", "ATM Withdrawal"], n_transactions
    )
})


In [15]:
customers.head()

,customer_id,age,gender,country,balance,account_type
0,1,56,Female,France,26892.91,Premium
1,2,69,Male,UK,45996.79,Basic
2,3,46,Male,UK,17349.98,Basic
3,4,32,Female,UK,17380.31,Premium
4,5,60,Male,Spain,36888.19,Basic


In [16]:
products.head()

,product_id,customer_id,product_name,product_category
0,1,1,Savings Account,Credit
1,2,2,Savings Account,Investment
2,3,3,Credit Card,Investment
3,4,4,Loan,Credit
4,5,5,Credit Card,Loan


In [24]:
transactions.head()

,transaction_id,customer_id,transaction_date,amount,transaction_type
0,1,301,2025-06-12,933.23,Transfer
1,2,90,2025-09-30,121.18,ATM Withdrawal
2,3,103,2025-07-06,528.24,Card Payment
3,4,476,2025-08-24,874.96,Transfer
4,5,250,2025-12-13,318.02,Transfer


In [25]:
customers.info

<bound method DataFrame.info of      customer_id  age  gender  country   balance account_type
0              1   56  Female   France  26892.91      Premium
1              2   69    Male       UK  45996.79        Basic
2              3   46    Male       UK  17349.98        Basic
3              4   32  Female       UK  17380.31      Premium
4              5   60    Male    Spain  36888.19        Basic
..           ...  ...     ...      ...       ...          ...
495          496   65    Male  Germany   3064.59        Basic
496          497   42  Female       UK  12392.81        Basic
497          498   57  Female   France   8019.26        Basic
498          499   62    Male   France  43595.59        Basic
499          500   18  Female    Spain  10999.74      Premium

[500 rows x 6 columns]>

In [21]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        500 non-null    int64 
 1   customer_id       500 non-null    int64 
 2   product_name      500 non-null    object
 3   product_category  500 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.8+ KB


In [26]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    8000 non-null   int64  
 1   customer_id       8000 non-null   int64  
 2   transaction_date  8000 non-null   object 
 3   amount            8000 non-null   float64
 4   transaction_type  8000 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 312.6+ KB


In [28]:
#change transaction_date to datetime
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])

In [32]:
print(transactions['transaction_date'].dtype)

datetime64[ns]


In [34]:
pip install pymysql sqlalchemy

In [37]:
#Connect to MySQL database
from sqlalchemy import create_engine

username = "root"
password = "Chioma07!"
host = "localhost"
port = "3306"
database ="Customer_Transaction_Behaviour"

#engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}")
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

customers.to_sql("customers", engine, if_exists="replace", index=False)
transactions.to_sql("transactions", engine, if_exists="replace", index=False)
products.to_sql("products", engine, if_exists="replace", index=False)



500